<a href="https://colab.research.google.com/github/EmilyHong77/gentrification_in_montreal/blob/main/notebooks/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Drive**

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Libraries**

In [69]:
import pandas as pd
import pprint
from collections import Counter

# **2001**
- merge population gender
- merge population age buckets
- merge no certificate or diploma
- merge regular maintenance
- merge transportation gender
- merge transportation others
- rename variables!

In [70]:
census_2001 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/2001_data_variables.csv')

# Print all column names
print("Column names:")
for col in census_2001.columns:
    print(col)


Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA01_7: 0-4
v_CA01_8: 5-9
v_CA01_9: 10-14
v_CA01_10: 15-19
v_CA01_11: 20-24
v_CA01_12: 25-29
v_CA01_13: 30-34
v_CA01_14: 35-39
v_CA01_15: 40-44
v_CA01_16: 45-49
v_CA01_17: 50-54
v_CA01_18: 55-59
v_CA01_19: 60-64
v_CA01_20: 65-69
v_CA01_21: 70-74
v_CA01_22: 75-79
v_CA01_23: 80-84
v_CA01_24: 85+
v_CA01_26: 0-4
v_CA01_27: 5-9
v_CA01_28: 10-14
v_CA01_29: 15-19
v_CA01_30: 20-24
v_CA01_31: 25-29
v_CA01_32: 30-34
v_CA01_33: 35-39
v_CA01_34: 40-44
v_CA01_35: 45-49
v_CA01_36: 50-54
v_CA01_37: 55-59
v_CA01_38: 60-64
v_CA01_39: 65-69
v_CA01_40: 70-74
v_CA01_41: 75-79
v_CA01_42: 80-84
v_CA01_43: 85+
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Intern

In [71]:
# Drop employment rate and unemployment rate
census_2001.drop(columns=['v_CA01_741: Employment rate', 'v_CA01_742: Unemployment rate'], inplace=True)

# Check
print("Column names:")
for col in census_2001.columns:
    print(col)


Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA01_7: 0-4
v_CA01_8: 5-9
v_CA01_9: 10-14
v_CA01_10: 15-19
v_CA01_11: 20-24
v_CA01_12: 25-29
v_CA01_13: 30-34
v_CA01_14: 35-39
v_CA01_15: 40-44
v_CA01_16: 45-49
v_CA01_17: 50-54
v_CA01_18: 55-59
v_CA01_19: 60-64
v_CA01_20: 65-69
v_CA01_21: 70-74
v_CA01_22: 75-79
v_CA01_23: 80-84
v_CA01_24: 85+
v_CA01_26: 0-4
v_CA01_27: 5-9
v_CA01_28: 10-14
v_CA01_29: 15-19
v_CA01_30: 20-24
v_CA01_31: 25-29
v_CA01_32: 30-34
v_CA01_33: 35-39
v_CA01_34: 40-44
v_CA01_35: 45-49
v_CA01_36: 50-54
v_CA01_37: 55-59
v_CA01_38: 60-64
v_CA01_39: 65-69
v_CA01_40: 70-74
v_CA01_41: 75-79
v_CA01_42: 80-84
v_CA01_43: 85+
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Intern

In [72]:
# Add employed and unemployed
emp_unemp = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/employed_unemployed_2001.csv')
census_2001 = pd.merge(census_2001, emp_unemp, on='GeoUID')

# Check
print("Column names:")
for col in census_2001.columns:
    print(col)


Column names:
GeoUID
Type_x
Region Name_x
Area (sq km)_x
Population _x
Dwellings _x
Households _x
v_CA01_7: 0-4
v_CA01_8: 5-9
v_CA01_9: 10-14
v_CA01_10: 15-19
v_CA01_11: 20-24
v_CA01_12: 25-29
v_CA01_13: 30-34
v_CA01_14: 35-39
v_CA01_15: 40-44
v_CA01_16: 45-49
v_CA01_17: 50-54
v_CA01_18: 55-59
v_CA01_19: 60-64
v_CA01_20: 65-69
v_CA01_21: 70-74
v_CA01_22: 75-79
v_CA01_23: 80-84
v_CA01_24: 85+
v_CA01_26: 0-4
v_CA01_27: 5-9
v_CA01_28: 10-14
v_CA01_29: 15-19
v_CA01_30: 20-24
v_CA01_31: 25-29
v_CA01_32: 30-34
v_CA01_33: 35-39
v_CA01_34: 40-44
v_CA01_35: 45-49
v_CA01_36: 50-54
v_CA01_37: 55-59
v_CA01_38: 60-64
v_CA01_39: 65-69
v_CA01_40: 70-74
v_CA01_41: 75-79
v_CA01_42: 80-84
v_CA01_43: 85+
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01

In [73]:
# Drop extras
census_2001 = census_2001.drop(columns=['Type_y', 'Region Name_y', 'Area (sq km)_y', 'Population _y', 'Dwellings _y', 'Households _y'])

print(census_2001.columns.tolist())

['GeoUID', 'Type_x', 'Region Name_x', 'Area (sq km)_x', 'Population _x', 'Dwellings _x', 'Households _x', 'v_CA01_7: 0-4', 'v_CA01_8: 5-9', 'v_CA01_9: 10-14', 'v_CA01_10: 15-19', 'v_CA01_11: 20-24', 'v_CA01_12: 25-29', 'v_CA01_13: 30-34', 'v_CA01_14: 35-39', 'v_CA01_15: 40-44', 'v_CA01_16: 45-49', 'v_CA01_17: 50-54', 'v_CA01_18: 55-59', 'v_CA01_19: 60-64', 'v_CA01_20: 65-69', 'v_CA01_21: 70-74', 'v_CA01_22: 75-79', 'v_CA01_23: 80-84', 'v_CA01_24: 85+', 'v_CA01_26: 0-4', 'v_CA01_27: 5-9', 'v_CA01_28: 10-14', 'v_CA01_29: 15-19', 'v_CA01_30: 20-24', 'v_CA01_31: 25-29', 'v_CA01_32: 30-34', 'v_CA01_33: 35-39', 'v_CA01_34: 40-44', 'v_CA01_35: 45-49', 'v_CA01_36: 50-54', 'v_CA01_37: 55-59', 'v_CA01_38: 60-64', 'v_CA01_39: 65-69', 'v_CA01_40: 70-74', 'v_CA01_41: 75-79', 'v_CA01_42: 80-84', 'v_CA01_43: 85+', 'v_CA01_6: Male, total', 'v_CA01_25: Female, total', 'v_CA01_3: Population percentage change, 1996-2001', 'v_CA01_45: Never legally married (single)', 'v_CA01_48: Divorced', 'v_CA01_49: Wid

In [74]:
# Remove white spaces
census_2001.columns = census_2001.columns.str.strip()

In [75]:
# Convert all columns except the identifier to numeric
ctuidcol = 'GeoUID'  # identifier column
cols_to_convert = census_2001.columns.difference([ctuidcol])
census_2001[cols_to_convert] = census_2001[cols_to_convert].apply(pd.to_numeric, errors='coerce')


In [76]:
# Merge population gender

census_2001['0-4'] = census_2001['v_CA01_7: 0-4'] + census_2001['v_CA01_26: 0-4']
census_2001 = census_2001.drop(columns=['v_CA01_7: 0-4', 'v_CA01_26: 0-4'])

census_2001['5-9'] = census_2001['v_CA01_8: 5-9'] + census_2001['v_CA01_27: 5-9']
census_2001 = census_2001.drop(columns=['v_CA01_8: 5-9', 'v_CA01_27: 5-9'])

census_2001['10-14'] = census_2001['v_CA01_9: 10-14'] + census_2001['v_CA01_28: 10-14']
census_2001 = census_2001.drop(columns=['v_CA01_9: 10-14', 'v_CA01_28: 10-14'])

census_2001['15-19'] = census_2001['v_CA01_10: 15-19'] + census_2001['v_CA01_29: 15-19']
census_2001 = census_2001.drop(columns=['v_CA01_10: 15-19', 'v_CA01_29: 15-19'])

census_2001['20-24'] = census_2001['v_CA01_11: 20-24'] + census_2001['v_CA01_30: 20-24']
census_2001 = census_2001.drop(columns=['v_CA01_11: 20-24', 'v_CA01_30: 20-24'])

census_2001['25-29'] = census_2001['v_CA01_12: 25-29'] + census_2001['v_CA01_31: 25-29']
census_2001 = census_2001.drop(columns=['v_CA01_12: 25-29', 'v_CA01_31: 25-29'])

census_2001['30-34'] = census_2001['v_CA01_13: 30-34'] + census_2001['v_CA01_32: 30-34']
census_2001 = census_2001.drop(columns=['v_CA01_13: 30-34', 'v_CA01_32: 30-34'])

census_2001['35-39'] = census_2001['v_CA01_14: 35-39'] + census_2001['v_CA01_33: 35-39']
census_2001 = census_2001.drop(columns=['v_CA01_14: 35-39', 'v_CA01_33: 35-39'])

census_2001['40-44'] = census_2001['v_CA01_15: 40-44'] + census_2001['v_CA01_34: 40-44']
census_2001 = census_2001.drop(columns=['v_CA01_15: 40-44', 'v_CA01_34: 40-44'])

census_2001['45-49'] = census_2001['v_CA01_16: 45-49'] + census_2001['v_CA01_35: 45-49']
census_2001 = census_2001.drop(columns=['v_CA01_16: 45-49', 'v_CA01_35: 45-49'])

census_2001['50-54'] = census_2001['v_CA01_17: 50-54'] + census_2001['v_CA01_36: 50-54']
census_2001 = census_2001.drop(columns=['v_CA01_17: 50-54', 'v_CA01_36: 50-54'])

census_2001['55-59'] = census_2001['v_CA01_18: 55-59'] + census_2001['v_CA01_37: 55-59']
census_2001 = census_2001.drop(columns=['v_CA01_18: 55-59', 'v_CA01_37: 55-59'])

census_2001['60-64'] = census_2001['v_CA01_19: 60-64'] + census_2001['v_CA01_38: 60-64']
census_2001 = census_2001.drop(columns=['v_CA01_19: 60-64', 'v_CA01_38: 60-64'])

census_2001['65-69'] = census_2001['v_CA01_20: 65-69'] + census_2001['v_CA01_39: 65-69']
census_2001 = census_2001.drop(columns=['v_CA01_20: 65-69', 'v_CA01_39: 65-69'])

census_2001['70-74'] = census_2001['v_CA01_21: 70-74'] + census_2001['v_CA01_40: 70-74']
census_2001 = census_2001.drop(columns=['v_CA01_21: 70-74', 'v_CA01_40: 70-74'])

census_2001['75-79'] = census_2001['v_CA01_22: 75-79'] + census_2001['v_CA01_41: 75-79']
census_2001 = census_2001.drop(columns=['v_CA01_22: 75-79', 'v_CA01_41: 75-79'])

census_2001['80-84'] = census_2001['v_CA01_23: 80-84'] + census_2001['v_CA01_42: 80-84']
census_2001 = census_2001.drop(columns=['v_CA01_23: 80-84', 'v_CA01_42: 80-84'])

census_2001['85+'] = census_2001['v_CA01_24: 85+'] + census_2001['v_CA01_43: 85+']
census_2001 = census_2001.drop(columns=['v_CA01_24: 85+', 'v_CA01_43: 85+'])

# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID
Type_x
Region Name_x
Area (sq km)_x
Population _x
Dwellings _x
Households _x
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Internal migrants
v_CA01_389: External migrants
v_CA01_703: Total visible minority population
v_CA01_716: All others
v_CA01_401: Citizenship other than Canadian
v_CA01_403: Non-immigrant population
v_CA01_406: Total immigrants by selected places of birth
v_CA01_458: Non-permanent residents
v_CA01_55: Married couples
v_CA01_61: Common-law couples
v_CA01_99: Owned
v_CA01_100: Rented
v_CA01_1634: Median household income $
v_CA01_1387: Without high school graduation certificate
v_CA01_1391: Without certificate or diploma
v_CA01_1394: Without degree
v_CA01_1388: With high school graduation

/tmp/ipython-input-834214814.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  census_2001['0-4'] = census_2001['v_CA01_7: 0-4'] + census_2001['v_CA01_26: 0-4']
/tmp/ipython-input-834214814.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  census_2001['5-9'] = census_2001['v_CA01_8: 5-9'] + census_2001['v_CA01_27: 5-9']
/tmp/ipython-input-834214814.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [77]:
# Merge population age buckets

census_2001['Population 0 to 9'] = census_2001['0-4'] + census_2001['5-9']
census_2001 = census_2001.drop(columns=['0-4', '5-9'])

census_2001['Population 10 to 19'] = census_2001['10-14'] + census_2001['15-19']
census_2001 = census_2001.drop(columns=['10-14', '15-19'])

census_2001['Population 20 to 29'] = census_2001['20-24'] + census_2001['25-29']
census_2001 = census_2001.drop(columns=['20-24', '25-29'])

census_2001['Population 30 to 39'] = census_2001['30-34'] + census_2001['35-39']
census_2001 = census_2001.drop(columns=['30-34', '35-39'])

census_2001['Population 40 to 49'] = census_2001['40-44'] + census_2001['45-49']
census_2001 = census_2001.drop(columns=['40-44', '45-49'])

census_2001['Population 50 to 59'] = census_2001['50-54'] + census_2001['55-59']
census_2001 = census_2001.drop(columns=['50-54', '55-59'])

census_2001['Population 60 plus'] = census_2001['60-64'] + census_2001['65-69'] + census_2001['70-74'] + census_2001['75-79'] + census_2001['80-84'] + census_2001['85+']
census_2001 = census_2001.drop(columns=['60-64', '65-69', '70-74', '75-79', '80-84', '85+'])



In [78]:
# Check values
census_2001.head(60)

,GeoUID,Type_x,Region Name_x,Area (sq km)_x,Population _x,Dwellings _x,Households _x,"v_CA01_6: Male, total","v_CA01_25: Female, total","v_CA01_3: Population percentage change, 1996-2001",...,"v_CA01_212: English, French and non-official language",v_CA01_737: Employed,v_CA01_738: Unemployed,Population 0 to 9,Population 10 to 19,Population 20 to 29,Population 30 to 39,Population 40 to 49,Population 50 to 59,Population 60 plus
0,4620001.00,NaN,NaN,0.4583,2445,1226,1165,1185.0,1265.0,2.4,...,0.0,1180.0,65.0,240.0,225.0,295.0,360.0,455.0,365.0,510.0
1,4620002.00,NaN,NaN,0.3865,2734,1339,1280,1280.0,1450.0,-2.8,...,15.0,1250.0,130.0,275.0,305.0,325.0,405.0,465.0,320.0,635.0
2,4620003.00,NaN,NaN,0.7392,6522,2945,2850,3055.0,3470.0,-0.1,...,0.0,3030.0,280.0,675.0,765.0,840.0,950.0,1150.0,905.0,1245.0
3,4620004.00,NaN,NaN,0.4516,3213,1575,1525,1505.0,1705.0,2.7,...,0.0,1505.0,115.0,270.0,335.0,385.0,435.0,590.0,445.0,760.0
4,4620005.00,NaN,NaN,0.5620,3171,1630,1565,1520.0,1655.0,-1.4,...,0.0,1535.0,120.0,265.0,305.0,370.0,430.0,575.0,495.0,735.0
5,4620006.00,NaN,NaN,0.6476,4541,2081,2015,2145.0,2395.0,3.7,...,0.0,2045.0,230.0,580.0,545.0,720.0,720.0,720.0,545.0,700.0
6,4620007.00,NaN,NaN,1.0993,5699,2849,2780,2595.0,3105.0,0.4,...,10.0,2670.0,320.0,505.0,535.0,710.0,775.0,975.0,815.0,1405.0
7,4620008.00,NaN,NaN,0.6239,2957,1441,1385,1425.0,1530.0,-0.3,...,0.0,1535.0,110.0,300.0,310.0,340.0,435.0,585.0,400.0,590.0
8,4620009.00,NaN,NaN,4.2700,3657,1669,1615,1805.0,1855.0,-3.9,...,0.0,1735.0,170.0,370.0,430.0,515.0,595.0,630.0,465.0,645.0
9,4620010.00,NaN,NaN,0.9323,1444,674,665,755.0,685.0,1.8,...,0.0,665.0,50.0,150.0,115.0,200.0,235.0,240.0,205.0,300.0


In [79]:
# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID
Type_x
Region Name_x
Area (sq km)_x
Population _x
Dwellings _x
Households _x
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Internal migrants
v_CA01_389: External migrants
v_CA01_703: Total visible minority population
v_CA01_716: All others
v_CA01_401: Citizenship other than Canadian
v_CA01_403: Non-immigrant population
v_CA01_406: Total immigrants by selected places of birth
v_CA01_458: Non-permanent residents
v_CA01_55: Married couples
v_CA01_61: Common-law couples
v_CA01_99: Owned
v_CA01_100: Rented
v_CA01_1634: Median household income $
v_CA01_1387: Without high school graduation certificate
v_CA01_1391: Without certificate or diploma
v_CA01_1394: Without degree
v_CA01_1388: With high school graduation

In [80]:
# Merge no certificate or diploma
census_2001['No certificate or diploma'] = census_2001['v_CA01_1387: Without high school graduation certificate']+ census_2001['v_CA01_1391: Without certificate or diploma'] + census_2001['v_CA01_1394: Without degree']
census_2001 = census_2001.drop(columns=['v_CA01_1387: Without high school graduation certificate', 'v_CA01_1391: Without certificate or diploma', 'v_CA01_1394: Without degree'])

# Merge regular maintenance
census_2001['Regular maintenance'] = census_2001['v_CA01_102: Regular maintenance only']+ census_2001['v_CA01_103: Minor repairs']
census_2001 = census_2001.drop(columns=['v_CA01_102: Regular maintenance only', 'v_CA01_103: Minor repairs'])

# Merge transportation gender and other methods
census_2001['Public transit'] = census_2001['v_CA01_1257: Public transit'] + census_2001['v_CA01_1266: Public transit']
census_2001 = census_2001.drop(columns=['v_CA01_1257: Public transit', 'v_CA01_1266: Public transit'])

census_2001['Walked'] = census_2001['v_CA01_1258: Walked'] + census_2001['v_CA01_1267: Walked']
census_2001 = census_2001.drop(columns=['v_CA01_1258: Walked', 'v_CA01_1267: Walked'])

census_2001['Bicycle'] = census_2001['v_CA01_1259: Bicycle'] + census_2001['v_CA01_1268: Bicycle']
census_2001 = census_2001.drop(columns=['v_CA01_1259: Bicycle', 'v_CA01_1268: Bicycle'])

census_2001['Transportation other methods'] = census_2001['v_CA01_1260: Motorcycle'] + census_2001['v_CA01_1261: Taxicab'] + census_2001['v_CA01_1262: Other method'] + census_2001['v_CA01_1269: Motorcycle'] + census_2001['v_CA01_1270: Taxicab'] + census_2001['v_CA01_1271: Other method']
census_2001 = census_2001.drop(columns=['v_CA01_1260: Motorcycle', 'v_CA01_1261: Taxicab', 'v_CA01_1262: Other method', 'v_CA01_1269: Motorcycle', 'v_CA01_1270: Taxicab', 'v_CA01_1271: Other method'])

census_2001['Transportation vehicle driver'] = census_2001['v_CA01_1255: Car, truck, van, as driver'] + census_2001['v_CA01_1264: Car, truck, van, as driver']
census_2001 = census_2001.drop(columns=['v_CA01_1255: Car, truck, van, as driver', 'v_CA01_1264: Car, truck, van, as driver'])

census_2001['Transportation vehicle non-driver'] = census_2001['v_CA01_1256: Car, truck, van, as passenger'] + census_2001['v_CA01_1265: Car, truck, van, as passenger']
census_2001 = census_2001.drop(columns=['v_CA01_1256: Car, truck, van, as passenger', 'v_CA01_1265: Car, truck, van, as passenger'])



In [81]:
# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID
Type_x
Region Name_x
Area (sq km)_x
Population _x
Dwellings _x
Households _x
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Internal migrants
v_CA01_389: External migrants
v_CA01_703: Total visible minority population
v_CA01_716: All others
v_CA01_401: Citizenship other than Canadian
v_CA01_403: Non-immigrant population
v_CA01_406: Total immigrants by selected places of birth
v_CA01_458: Non-permanent residents
v_CA01_55: Married couples
v_CA01_61: Common-law couples
v_CA01_99: Owned
v_CA01_100: Rented
v_CA01_1634: Median household income $
v_CA01_1388: With high school graduation certificate
v_CA01_1392: With certificate or diploma
v_CA01_1389: Trades certificate or diploma
v_CA01_1397: With bachelor's de

In [82]:
# Drop useless
census_2001 = census_2001.drop(columns=['Type_x', 'Region Name_x'])

# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID
Area (sq km)_x
Population _x
Dwellings _x
Households _x
v_CA01_6: Male, total
v_CA01_25: Female, total
v_CA01_3: Population percentage change, 1996-2001
v_CA01_45: Never legally married (single)
v_CA01_48: Divorced
v_CA01_49: Widowed
v_CA01_383: Movers
v_CA01_382: Non-movers
v_CA01_400: Canadian citizenship
v_CA01_385: Migrants
v_CA01_384: Non-migrants
v_CA01_386: Internal migrants
v_CA01_389: External migrants
v_CA01_703: Total visible minority population
v_CA01_716: All others
v_CA01_401: Citizenship other than Canadian
v_CA01_403: Non-immigrant population
v_CA01_406: Total immigrants by selected places of birth
v_CA01_458: Non-permanent residents
v_CA01_55: Married couples
v_CA01_61: Common-law couples
v_CA01_99: Owned
v_CA01_100: Rented
v_CA01_1634: Median household income $
v_CA01_1388: With high school graduation certificate
v_CA01_1392: With certificate or diploma
v_CA01_1389: Trades certificate or diploma
v_CA01_1397: With bachelor's degree or higher
v_CA01

In [83]:
print(len(census_2001.columns))

68


In [84]:
# Rename
census_2001["v_CA01_1397: With bachelors degree or higher"] = census_2001["v_CA01_1397: With bachelor's degree or higher"]
census_2001 = census_2001.drop(columns=["v_CA01_1397: With bachelor's degree or higher"])

print(census_2001.columns.tolist())


['GeoUID', 'Area (sq km)_x', 'Population _x', 'Dwellings _x', 'Households _x', 'v_CA01_6: Male, total', 'v_CA01_25: Female, total', 'v_CA01_3: Population percentage change, 1996-2001', 'v_CA01_45: Never legally married (single)', 'v_CA01_48: Divorced', 'v_CA01_49: Widowed', 'v_CA01_383: Movers', 'v_CA01_382: Non-movers', 'v_CA01_400: Canadian citizenship', 'v_CA01_385: Migrants', 'v_CA01_384: Non-migrants', 'v_CA01_386: Internal migrants', 'v_CA01_389: External migrants', 'v_CA01_703: Total visible minority population', 'v_CA01_716: All others', 'v_CA01_401: Citizenship other than Canadian', 'v_CA01_403: Non-immigrant population', 'v_CA01_406: Total immigrants by selected places of birth', 'v_CA01_458: Non-permanent residents', 'v_CA01_55: Married couples', 'v_CA01_61: Common-law couples', 'v_CA01_99: Owned', 'v_CA01_100: Rented', 'v_CA01_1634: Median household income $', 'v_CA01_1388: With high school graduation certificate', 'v_CA01_1392: With certificate or diploma', 'v_CA01_1389: T

In [85]:
print(census_2001['v_CA01_1397: With bachelors degree or higher'])

0        70.0
1        95.0
2       350.0
3       195.0
4       210.0
        ...  
857     230.0
858    1035.0
859     755.0
860     640.0
861    1175.0
Name: v_CA01_1397: With bachelors degree or higher, Length: 862, dtype: float64


In [86]:
# Rename variables
census_2001 = census_2001.rename(columns=
  {'Area (sq km)_x': 'Area (sq km)', 'Population _x': 'Population', 'Dwellings _x': 'Dwellings',
   'Households _x': 'Households', 'v_CA01_6: Male, total': 'Total male population', 'v_CA01_25: Female, total': 'Total female population',
   'v_CA01_3: Population percentage change, 1996-2001': 'Population change (%)', 'v_CA01_45: Never legally married (single)': 'Never married', 'v_CA01_48: Divorced': 'Divorced',
   'v_CA01_49: Widowed': 'Widowed', 'v_CA01_383: Movers': 'Movers', 'v_CA01_382: Non-movers': 'Non-movers',
   'v_CA01_400: Canadian citizenship': 'Canadian citizens', 'v_CA01_385: Migrants': 'Migrants', 'v_CA01_384: Non-migrants': 'Non-migrants',
   'v_CA01_386: Internal migrants': 'Internal migrants', 'v_CA01_389: External migrants': 'External migrants', 'v_CA01_703: Total visible minority population': 'Visible minority',
   'v_CA01_716: All others': 'Non-visible minority', 'v_CA01_401: Citizenship other than Canadian': 'Non-Canadian citizens', 'v_CA01_403: Non-immigrant population': 'Non-immigrants',
   'v_CA01_406: Total immigrants by selected places of birth': 'Immigrants', 'v_CA01_458: Non-permanent residents': 'Non-permanent residents', 'v_CA01_55: Married couples': 'Married couples',
   'v_CA01_61: Common-law couples': 'Common-law couples', 'v_CA01_99: Owned': 'Owned housing', 'v_CA01_100: Rented': 'Rented housing',
   'v_CA01_1634: Median household income $': 'Median household income $', 'v_CA01_1388: With high school graduation certificate': 'High school or Secondary degree', 'v_CA01_1392: With certificate or diploma': 'College or CEGEP degree',
   'v_CA01_1389: Trades certificate or diploma': 'Trades certificate, diploma or apprenticeship', 'v_CA01_1397: With bachelors degree or higher': 'Bachelors degree or higher', 'v_CA01_118: Apartment, building that has fewer than five storeys': 'Apartment with fewer than five stories',
   'v_CA01_117: Apartment, building that has five or more storeys': 'Apartment with five or more storeys', 'v_CA01_115: Row house': 'Row house', 'v_CA01_113: Single-detached house': 'Single-detached house',
   'v_CA01_114: Semi-detached house': 'Semi-detached house', 'v_CA01_120: Movable dwelling': 'Movable dwelling', 'v_CA01_119: Other single-attached house': 'Other single-attached house',
   'v_CA01_104: Major repairs': 'Major repairs', 'v_CA01_101: Band housing': 'Band housing', 'v_CA01_135: English': 'English only',
   'v_CA01_136: French': 'French only', 'v_CA01_137: Non-official languages': 'Allophone ', 'v_CA01_1667: Average gross rent $': 'Average gross rent ($)',
   'v_CA01_1674: Average value of dwelling $': 'Average value dwelling ($)', 'v_CA01_209: English and French': 'English and French', 'v_CA01_210: English and non-official language': 'English and non-official language(s)',
   'v_CA01_211: French and non-official language': 'French and non-official language(s)', 'v_CA01_212: English, French and non-official language': 'English, French, and non-official language(s)', 'v_CA01_737: Employed': 'Employed',
   'v_CA01_738: Unemployed': 'Unemployed'})



In [87]:
# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID
Area (sq km)
Population
Dwellings
Households
Total male population
Total female population
Population change (%)
Never married
Divorced
Widowed
Movers
Non-movers
Canadian citizens
Migrants
Non-migrants
Internal migrants
External migrants
Visible minority
Non-visible minority
Non-Canadian citizens
Non-immigrants
Immigrants
Non-permanent residents
Married couples
Common-law couples
Owned housing
Rented housing
Median household income $
High school or Secondary degree
College or CEGEP degree
Trades certificate, diploma or apprenticeship
Apartment with fewer than five stories
Apartment with five or more storeys
Row house
Single-detached house
Semi-detached house
Movable dwelling
Other single-attached house
Major repairs
Band housing
English only
French only
Allophone 
Average gross rent ($)
Average value dwelling ($)
English and French
English and non-official language(s)
French and non-official language(s)
English, French, and non-official language(s)
Employed
Unemplo

In [88]:
print(len(census_2001.columns))

68


In [89]:
# Add year
census_2001 = census_2001.add_suffix('_2001')

# Check
print("Column names:")
for col in census_2001.columns:
    print(col)

Column names:
GeoUID_2001
Area (sq km)_2001
Population_2001
Dwellings_2001
Households_2001
Total male population_2001
Total female population_2001
Population change (%)_2001
Never married_2001
Divorced_2001
Widowed_2001
Movers_2001
Non-movers_2001
Canadian citizens_2001
Migrants_2001
Non-migrants_2001
Internal migrants_2001
External migrants_2001
Visible minority_2001
Non-visible minority_2001
Non-Canadian citizens_2001
Non-immigrants_2001
Immigrants_2001
Non-permanent residents_2001
Married couples_2001
Common-law couples_2001
Owned housing_2001
Rented housing_2001
Median household income $_2001
High school or Secondary degree_2001
College or CEGEP degree_2001
Trades certificate, diploma or apprenticeship_2001
Apartment with fewer than five stories_2001
Apartment with five or more storeys_2001
Row house_2001
Single-detached house_2001
Semi-detached house_2001
Movable dwelling_2001
Other single-attached house_2001
Major repairs_2001
Band housing_2001
English only_2001
French only_2001


In [93]:
# Check values
census_2001.iloc[:60, 40:60]


,Band housing_2001,English only_2001,French only_2001,Allophone _2001,Average gross rent ($)_2001,Average value dwelling ($)_2001,English and French_2001,English and non-official language(s)_2001,French and non-official language(s)_2001,"English, French, and non-official language(s)_2001",Employed_2001,Unemployed_2001,Population 0 to 9_2001,Population 10 to 19_2001,Population 20 to 29_2001,Population 30 to 39_2001,Population 40 to 49_2001,Population 50 to 59_2001,Population 60 plus_2001,No certificate or diploma_2001
0,0.0,30.0,2260.0,95.0,554.0,107148.0,20.0,0.0,10.0,0.0,1180.0,65.0,240.0,225.0,295.0,360.0,455.0,365.0,510.0,555.0
1,0.0,20.0,2470.0,190.0,528.0,107098.0,10.0,0.0,15.0,15.0,1250.0,130.0,275.0,305.0,325.0,405.0,465.0,320.0,635.0,615.0
2,0.0,205.0,5215.0,930.0,541.0,139439.0,35.0,20.0,35.0,0.0,3030.0,280.0,675.0,765.0,840.0,950.0,1150.0,905.0,1245.0,1385.0
3,0.0,105.0,2800.0,165.0,533.0,129464.0,20.0,0.0,30.0,0.0,1505.0,115.0,270.0,335.0,385.0,435.0,590.0,445.0,760.0,550.0
4,0.0,40.0,2900.0,100.0,504.0,120973.0,25.0,0.0,0.0,0.0,1535.0,120.0,265.0,305.0,370.0,430.0,575.0,495.0,735.0,635.0
5,0.0,150.0,3910.0,330.0,504.0,132531.0,25.0,20.0,35.0,0.0,2045.0,230.0,580.0,545.0,720.0,720.0,720.0,545.0,700.0,1100.0
6,0.0,185.0,4735.0,670.0,481.0,115460.0,35.0,10.0,20.0,10.0,2670.0,320.0,505.0,535.0,710.0,775.0,975.0,815.0,1405.0,1210.0
7,0.0,65.0,2790.0,85.0,502.0,111571.0,0.0,0.0,0.0,0.0,1535.0,110.0,300.0,310.0,340.0,435.0,585.0,400.0,590.0,595.0
8,0.0,90.0,3160.0,340.0,499.0,107901.0,10.0,0.0,15.0,0.0,1735.0,170.0,370.0,430.0,515.0,595.0,630.0,465.0,645.0,730.0
9,0.0,20.0,1360.0,40.0,554.0,114200.0,10.0,10.0,0.0,0.0,665.0,50.0,150.0,115.0,200.0,235.0,240.0,205.0,300.0,275.0


In [94]:
# Export file to drive
census_2001.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2001_clean.csv', index=False)

# **2006**
- merge population gender
- merge population age buckets
- calculate pop percentage change and add
- merge regular maintenance
- merge transporation others
- rename variables!

In [117]:
census_2006 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/2006_data_variables.csv')

# Print all column names
print("Column names:")
for col in census_2006.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA06_4: 0 to 4 years
v_CA06_5: 5 to 9 years
v_CA06_6: 10 to 14 years
v_CA06_7: 15 to 19 years
v_CA06_8: 20 to 24 years
v_CA06_9: 25 to 29 years
v_CA06_10: 30 to 34 years
v_CA06_11: 35 to 39 years
v_CA06_12: 40 to 44 years
v_CA06_13: 45 to 49 years
v_CA06_14: 50 to 54 years
v_CA06_15: 55 to 59 years
v_CA06_16: 60 to 64 years
v_CA06_17: 65 to 69 years
v_CA06_18: 70 to 74 years
v_CA06_19: 75 to 79 years
v_CA06_20: 80 to 84 years
v_CA06_21: 85 years and over
v_CA06_23: 0 to 4 years
v_CA06_24: 5 to 9 years
v_CA06_25: 10 to 14 years
v_CA06_26: 15 to 19 years
v_CA06_27: 20 to 24 years
v_CA06_28: 25 to 29 years
v_CA06_29: 30 to 34 years
v_CA06_30: 35 to 39 years
v_CA06_31: 40 to 44 years
v_CA06_32: 45 to 49 years
v_CA06_33: 50 to 54 years
v_CA06_34: 55 to 59 years
v_CA06_35: 60 to 64 years
v_CA06_36: 65 to 69 years
v_CA06_37: 70 to 74 years
v_CA06_38: 75 to 79 years
v_CA06_39: 80 to 84 years
v_CA06_40: 85 y

In [118]:
# Check values
census_2001.head(60)

,GeoUID_2001,Area (sq km)_2001,Population_2001,Dwellings_2001,Households_2001,Total male population_2001,Total female population_2001,Population change (%)_2001,Never married_2001,Divorced_2001,...,Population 60 plus_2001,No certificate or diploma_2001,Regular maintenance_2001,Public transit_2001,Walked_2001,Bicycle_2001,Transportation other methods_2001,Transportation vehicle driver_2001,Transportation vehicle non-driver_2001,Bachelors degree or higher_2001
0,4620001.00,0.4583,2445,1226,1165,1185.0,1265.0,2.4,965.0,270.0,...,510.0,555.0,1055.0,315.0,30.0,40.0,0.0,695.0,40.0,70.0
1,4620002.00,0.3865,2734,1339,1280,1280.0,1450.0,-2.8,995.0,285.0,...,635.0,615.0,1180.0,395.0,45.0,15.0,0.0,705.0,40.0,95.0
2,4620003.00,0.7392,6522,2945,2850,3055.0,3470.0,-0.1,2255.0,675.0,...,1245.0,1385.0,2645.0,840.0,155.0,10.0,10.0,1830.0,80.0,350.0
3,4620004.00,0.4516,3213,1575,1525,1505.0,1705.0,2.7,1180.0,355.0,...,760.0,550.0,1425.0,430.0,80.0,10.0,0.0,870.0,45.0,195.0
4,4620005.00,0.5620,3171,1630,1565,1520.0,1655.0,-1.4,1200.0,425.0,...,735.0,635.0,1440.0,510.0,70.0,35.0,0.0,815.0,80.0,210.0
5,4620006.00,0.6476,4541,2081,2015,2145.0,2395.0,3.7,1875.0,450.0,...,700.0,1100.0,1850.0,675.0,110.0,30.0,20.0,1075.0,70.0,265.0
6,4620007.00,1.0993,5699,2849,2780,2595.0,3105.0,0.4,2025.0,620.0,...,1405.0,1210.0,2575.0,1050.0,205.0,20.0,10.0,1205.0,80.0,555.0
7,4620008.00,0.6239,2957,1441,1385,1425.0,1530.0,-0.3,1125.0,305.0,...,590.0,595.0,1275.0,350.0,40.0,10.0,20.0,1045.0,35.0,320.0
8,4620009.00,4.2700,3657,1669,1615,1805.0,1855.0,-3.9,1485.0,340.0,...,645.0,730.0,1405.0,540.0,95.0,25.0,20.0,1000.0,55.0,160.0
9,4620010.00,0.9323,1444,674,665,755.0,685.0,1.8,590.0,145.0,...,300.0,275.0,585.0,210.0,65.0,30.0,10.0,340.0,0.0,65.0


In [119]:
# Convert all columns except the identifier to numeric
ctuidcol = 'GeoUID'  # identifier column
cols_to_convert = census_2006.columns.difference([ctuidcol])
census_2006[cols_to_convert] = census_2006[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [120]:
# Merge population gender and age buckets
census_2006['Population 0 to 9'] = census_2006['v_CA06_4: 0 to 4 years'] + census_2006['v_CA06_23: 0 to 4 years'] + census_2006['v_CA06_5: 5 to 9 years'] + census_2006['v_CA06_24: 5 to 9 years']
census_2006 = census_2006.drop(columns=['v_CA06_4: 0 to 4 years', 'v_CA06_23: 0 to 4 years', 'v_CA06_5: 5 to 9 years', 'v_CA06_24: 5 to 9 years'])

census_2006['Population 10 to 19'] = census_2006['v_CA06_6: 10 to 14 years'] + census_2006['v_CA06_25: 10 to 14 years'] + census_2006['v_CA06_7: 15 to 19 years'] + census_2006['v_CA06_26: 15 to 19 years']
census_2006 = census_2006.drop(columns=['v_CA06_6: 10 to 14 years', 'v_CA06_25: 10 to 14 years', 'v_CA06_7: 15 to 19 years', 'v_CA06_26: 15 to 19 years'])

census_2006['Population 20 to 29'] = census_2006['v_CA06_8: 20 to 24 years'] + census_2006['v_CA06_27: 20 to 24 years'] + census_2006['v_CA06_9: 25 to 29 years'] + census_2006['v_CA06_28: 25 to 29 years']
census_2006 = census_2006.drop(columns=['v_CA06_8: 20 to 24 years', 'v_CA06_27: 20 to 24 years', 'v_CA06_9: 25 to 29 years', 'v_CA06_28: 25 to 29 years'])

census_2006['Population 30 to 39'] = census_2006['v_CA06_10: 30 to 34 years'] + census_2006['v_CA06_29: 30 to 34 years'] + census_2006['v_CA06_11: 35 to 39 years'] + census_2006['v_CA06_30: 35 to 39 years']
census_2006 = census_2006.drop(columns=['v_CA06_10: 30 to 34 years', 'v_CA06_29: 30 to 34 years', 'v_CA06_11: 35 to 39 years', 'v_CA06_30: 35 to 39 years'])

census_2006['Population 40 to 49'] = census_2006['v_CA06_12: 40 to 44 years'] + census_2006['v_CA06_31: 40 to 44 years'] + census_2006['v_CA06_13: 45 to 49 years'] + census_2006['v_CA06_32: 45 to 49 years']
census_2006 = census_2006.drop(columns=['v_CA06_12: 40 to 44 years', 'v_CA06_31: 40 to 44 years', 'v_CA06_13: 45 to 49 years', 'v_CA06_32: 45 to 49 years'])

census_2006['Population 50 to 59'] = census_2006['v_CA06_14: 50 to 54 years'] + census_2006['v_CA06_33: 50 to 54 years'] + census_2006['v_CA06_15: 55 to 59 years'] + census_2006['v_CA06_34: 55 to 59 years']
census_2006 = census_2006.drop(columns=['v_CA06_14: 50 to 54 years', 'v_CA06_33: 50 to 54 years', 'v_CA06_15: 55 to 59 years', 'v_CA06_34: 55 to 59 years'])

census_2006['Population 60 plus'] = census_2006['v_CA06_16: 60 to 64 years'] + census_2006['v_CA06_35: 60 to 64 years'] + census_2006['v_CA06_17: 65 to 69 years'] + census_2006['v_CA06_36: 65 to 69 years'] + census_2006['v_CA06_18: 70 to 74 years'] + census_2006['v_CA06_37: 70 to 74 years'] + census_2006['v_CA06_19: 75 to 79 years'] + census_2006['v_CA06_38: 75 to 79 years'] + census_2006['v_CA06_20: 80 to 84 years'] + census_2006['v_CA06_39: 80 to 84 years'] + census_2006['v_CA06_21: 85 years and over'] + census_2006['v_CA06_40: 85 years and over']
census_2006 = census_2006.drop(columns=['v_CA06_16: 60 to 64 years', 'v_CA06_35: 60 to 64 years', 'v_CA06_17: 65 to 69 years', 'v_CA06_36: 65 to 69 years', 'v_CA06_18: 70 to 74 years', 'v_CA06_37: 70 to 74 years', 'v_CA06_19: 75 to 79 years', 'v_CA06_38: 75 to 79 years', 'v_CA06_20: 80 to 84 years', 'v_CA06_39: 80 to 84 years', 'v_CA06_21: 85 years and over', 'v_CA06_40: 85 years and over'])

# print all column names
print("Column names:")
for col in census_2006.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA06_3: Male, total
v_CA06_22: Female, total
v_CA06_42: Never legally married (single)
v_CA06_45: Divorced
v_CA06_46: Widowed
v_CA06_452: Non-movers
v_CA06_453: Movers
v_CA06_473: Not Canadian citizens
v_CA06_470: Canadian citizens
v_CA06_454: Non-migrants
v_CA06_455: Migrants
v_CA06_456: Internal migrants
v_CA06_459: External migrants
v_CA06_475: Non-immigrants
v_CA06_478: Immigrants
v_CA06_1303: Total visible minority population
v_CA06_1316: Not a visible minority
v_CA06_511: Non-permanent residents
v_CA06_63: Common-law couples
v_CA06_57: Married couples
v_CA06_2000: Median household income $
v_CA06_577: Employed
v_CA06_578: Unemployed
v_CA06_1249: No certificate, diploma or degree
v_CA06_1251: High school certificate or equivalent
v_CA06_1253: College, CEGEP or other non-university certificate or diploma
v_CA06_1252: Apprenticeship or trades certificate or diploma
v_CA06_1256: University certifi

/tmp/ipython-input-2683118502.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  census_2006['Population 0 to 9'] = census_2006['v_CA06_4: 0 to 4 years'] + census_2006['v_CA06_23: 0 to 4 years'] + census_2006['v_CA06_5: 5 to 9 years'] + census_2006['v_CA06_24: 5 to 9 years']


In [121]:
# Check values
census_2006.head(60)

,GeoUID,Type,Region Name,Area (sq km),Population,Dwellings,Households,"v_CA06_3: Male, total","v_CA06_22: Female, total",v_CA06_42: Never legally married (single),...,v_CA06_240: English and non-official language,v_CA06_241: French and non-official language,"v_CA06_242: English, French and non-official language",Population 0 to 9,Population 10 to 19,Population 20 to 29,Population 30 to 39,Population 40 to 49,Population 50 to 59,Population 60 plus
0,4620001.00,NaN,NaN,0.45831,2498,1306,1243,1215.0,1285.0,1085.0,...,0.0,10.0,0.0,205.0,240.0,325.0,345.0,480.0,380.0,520.0
1,4620002.00,NaN,NaN,0.38654,2819,1399,1343,1345.0,1475.0,1140.0,...,0.0,15.0,0.0,250.0,340.0,375.0,395.0,460.0,375.0,620.0
2,4620003.00,NaN,NaN,0.73920,6407,2969,2884,3030.0,3375.0,2495.0,...,10.0,40.0,10.0,585.0,715.0,860.0,785.0,1090.0,1010.0,1375.0
3,4620004.00,NaN,NaN,0.45157,3223,1627,1559,1515.0,1710.0,1335.0,...,0.0,50.0,10.0,225.0,335.0,430.0,375.0,555.0,510.0,785.0
4,4620005.00,NaN,NaN,0.56196,3188,1664,1601,1535.0,1650.0,1380.0,...,0.0,0.0,10.0,245.0,310.0,400.0,430.0,580.0,515.0,725.0
5,4620006.00,NaN,NaN,0.64760,4568,2092,2013,2190.0,2380.0,2000.0,...,0.0,40.0,0.0,545.0,580.0,730.0,645.0,785.0,595.0,685.0
6,4620007.00,NaN,NaN,1.09934,5712,2894,2807,2635.0,3070.0,2265.0,...,0.0,10.0,0.0,430.0,570.0,760.0,665.0,975.0,905.0,1390.0
7,4620008.00,NaN,NaN,0.62386,2973,1467,1424,1470.0,1505.0,1330.0,...,0.0,25.0,0.0,245.0,345.0,365.0,375.0,580.0,495.0,590.0
8,4620009.00,NaN,NaN,4.27007,3554,1696,1643,1775.0,1785.0,1590.0,...,0.0,25.0,0.0,305.0,405.0,545.0,480.0,630.0,555.0,655.0
9,4620010.00,NaN,NaN,0.93219,1351,669,643,700.0,645.0,590.0,...,0.0,15.0,0.0,120.0,105.0,200.0,175.0,230.0,215.0,310.0


In [122]:
# Merge regular maintenance
census_2006['Regular maintenance'] = census_2006['v_CA06_106: Regular maintenance only']+ census_2006['v_CA06_107: Minor repairs']
census_2006 = census_2006.drop(columns=['v_CA06_106: Regular maintenance only', 'v_CA06_107: Minor repairs'])

# Merge transportion others
census_2006['Transportation other methods'] = census_2006['v_CA06_1106: Motorcycle'] + census_2006['v_CA06_1107: Taxicab'] + census_2006['v_CA06_1108: Other method']
census_2006 = census_2006.drop(columns=['v_CA06_1106: Motorcycle', 'v_CA06_1107: Taxicab', 'v_CA06_1108: Other method'])

# Print all column names
print("Column names:")
for col in census_2006.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA06_3: Male, total
v_CA06_22: Female, total
v_CA06_42: Never legally married (single)
v_CA06_45: Divorced
v_CA06_46: Widowed
v_CA06_452: Non-movers
v_CA06_453: Movers
v_CA06_473: Not Canadian citizens
v_CA06_470: Canadian citizens
v_CA06_454: Non-migrants
v_CA06_455: Migrants
v_CA06_456: Internal migrants
v_CA06_459: External migrants
v_CA06_475: Non-immigrants
v_CA06_478: Immigrants
v_CA06_1303: Total visible minority population
v_CA06_1316: Not a visible minority
v_CA06_511: Non-permanent residents
v_CA06_63: Common-law couples
v_CA06_57: Married couples
v_CA06_2000: Median household income $
v_CA06_577: Employed
v_CA06_578: Unemployed
v_CA06_1249: No certificate, diploma or degree
v_CA06_1251: High school certificate or equivalent
v_CA06_1253: College, CEGEP or other non-university certificate or diploma
v_CA06_1252: Apprenticeship or trades certificate or diploma
v_CA06_1256: University certifi

In [123]:
 # Calculate pop percentage change and add
c_2001 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2001_clean.csv')
census_2006['Population change (%)'] = ((census_2006['Population '] - c_2001['Population_2001']) / c_2001['Population_2001'] * 100).round(1)

print(census_2006['Population change (%)'])



0      2.2
1      3.1
2     -1.8
3      0.3
4      0.5
      ... 
873    NaN
874    NaN
875    NaN
876    NaN
877    NaN
Name: Population change (%), Length: 878, dtype: float64


In [124]:
# Check
print("Column names:")
for col in census_2006.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA06_3: Male, total
v_CA06_22: Female, total
v_CA06_42: Never legally married (single)
v_CA06_45: Divorced
v_CA06_46: Widowed
v_CA06_452: Non-movers
v_CA06_453: Movers
v_CA06_473: Not Canadian citizens
v_CA06_470: Canadian citizens
v_CA06_454: Non-migrants
v_CA06_455: Migrants
v_CA06_456: Internal migrants
v_CA06_459: External migrants
v_CA06_475: Non-immigrants
v_CA06_478: Immigrants
v_CA06_1303: Total visible minority population
v_CA06_1316: Not a visible minority
v_CA06_511: Non-permanent residents
v_CA06_63: Common-law couples
v_CA06_57: Married couples
v_CA06_2000: Median household income $
v_CA06_577: Employed
v_CA06_578: Unemployed
v_CA06_1249: No certificate, diploma or degree
v_CA06_1251: High school certificate or equivalent
v_CA06_1253: College, CEGEP or other non-university certificate or diploma
v_CA06_1252: Apprenticeship or trades certificate or diploma
v_CA06_1256: University certifi

In [125]:
# Rename variables!
census_2006 = census_2006.rename(columns=
  {'v_CA06_3: Male, total': 'Total male population', 'v_CA06_22: Female, total': 'Total female population',
   'v_CA06_42: Never legally married (single)': 'Never married', 'v_CA06_45: Divorced': 'Divorced',
   'v_CA06_46: Widowed': 'Widowed', 'v_CA06_453: Movers': 'Movers', 'v_CA06_452: Non-movers': 'Non-movers',
   'v_CA06_470: Canadian citizens': 'Canadian citizens',
   'v_CA06_455: Migrants': 'Migrants',
   'v_CA06_454: Non-migrants': 'Non-migrants',
   'v_CA06_456: Internal migrants': 'Internal migrants',
   'v_CA06_459: External migrants': 'External migrants',
   'v_CA06_1303: Total visible minority population': 'Visible minority',
   'v_CA06_1316: Not a visible minority': 'Non-visible minority',
   'v_CA06_473: Not Canadian citizens': 'Non-Canadian citizens',
   'v_CA06_475: Non-immigrants': 'Non-immigrants',
   'v_CA06_478: Immigrants': 'Immigrants',
   'v_CA06_511: Non-permanent residents': 'Non-permanent residents',
   'v_CA06_57: Married couples': 'Married couples',
   'v_CA06_63: Common-law couples': 'Common-law couples',
   'v_CA06_102: Owned': 'Owned housing',
   'v_CA06_103: Rented': 'Rented housing',
   'v_CA06_1249: No certificate, diploma or degree': 'No certificate or diploma',
   'v_CA06_2000: Median household income $': 'Median household income $',
   'v_CA06_1251: High school certificate or equivalent': 'High school or Secondary degree',
   'v_CA06_1253: College, CEGEP or other non-university certificate or diploma': 'College or CEGEP degree',
   'v_CA06_1252: Apprenticeship or trades certificate or diploma': 'Trades certificate, diploma or apprenticeship',
   'v_CA06_1256: University certificate or degree': 'Bachelors degree or higher',
   'v_CA06_125: Apartment, building that has fewer than five storeys': 'Apartment with fewer than five stories',
   'v_CA06_124: Apartment, building that has five or more storeys': 'Apartment with five or more storeys',
   'v_CA06_122: Row house': 'Row house',
   'v_CA06_120: Single-detached house': 'Single-detached house',
   'v_CA06_121: Semi-detached house': 'Semi-detached house',
   'v_CA06_127: Movable dwelling': 'Movable dwelling',
   'v_CA06_126: Other single-attached house': 'Other single-attached house',
   'v_CA06_108: Major repairs': 'Major repairs',
   'v_CA06_104: Band housing': 'Band housing',
   'v_CA06_142: English': 'English only',
   'v_CA06_143: French': 'French only',
   'v_CA06_144: Non-official languages': 'Allophone ',
   'v_CA06_2050: Average gross rent $': 'Average gross rent ($)',
   'v_CA06_2054: Average value of dwelling $': 'Average value dwelling ($)',
   'v_CA06_1103: Public transit' : 'Public transit',
   'v_CA06_1104: Walked' : 'Walked',
   'v_CA06_1105: Bicycle' : 'Bicycle',
   'v_CA06_1101: Car, truck, van, as driver': 'Transportation vehicle driver',
   'v_CA06_1102: Car, truck, van, as passenger' : 'Transportation vehicle non-driver',
   'v_CA06_239: English and French': 'English and French',
   'v_CA06_240: English and non-official language': 'English and non-official language(s)',
   'v_CA06_241: French and non-official language': 'French and non-official language(s)',
   'v_CA06_242: English, French and non-official language': 'English, French, and non-official language(s)',
   'v_CA06_577: Employed': 'Employed',
   'v_CA06_578: Unemployed': 'Unemployed'})


In [126]:
# Check
print("Column names:")
for col in census_2006.columns:
    print(col)

print(len(census_2006.columns))

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
Total male population
Total female population
Never married
Divorced
Widowed
Non-movers
Movers
Non-Canadian citizens
Canadian citizens
Non-migrants
Migrants
Internal migrants
External migrants
Non-immigrants
Immigrants
Visible minority
Non-visible minority
Non-permanent residents
Common-law couples
Married couples
Median household income $
Employed
Unemployed
No certificate or diploma
High school or Secondary degree
College or CEGEP degree
Trades certificate, diploma or apprenticeship
Bachelors degree or higher
Apartment with fewer than five stories
Apartment with five or more storeys
Row house
Single-detached house
Semi-detached house
Movable dwelling
Other single-attached house
Major repairs
Owned housing
Rented housing
Band housing
English only
French only
Allophone 
Average gross rent ($)
Average value dwelling ($)
Public transit
Walked
Bicycle
Transportation vehicle driver
Transportation vehicle 

In [127]:
# Drop extras
census_2006 = census_2006.drop(columns=['Type', 'Region Name'])

In [128]:
# Check
print("Column names:")
for col in census_2006.columns:
    print(col)

print(len(census_2006.columns))

Column names:
GeoUID
Area (sq km)
Population 
Dwellings 
Households 
Total male population
Total female population
Never married
Divorced
Widowed
Non-movers
Movers
Non-Canadian citizens
Canadian citizens
Non-migrants
Migrants
Internal migrants
External migrants
Non-immigrants
Immigrants
Visible minority
Non-visible minority
Non-permanent residents
Common-law couples
Married couples
Median household income $
Employed
Unemployed
No certificate or diploma
High school or Secondary degree
College or CEGEP degree
Trades certificate, diploma or apprenticeship
Bachelors degree or higher
Apartment with fewer than five stories
Apartment with five or more storeys
Row house
Single-detached house
Semi-detached house
Movable dwelling
Other single-attached house
Major repairs
Owned housing
Rented housing
Band housing
English only
French only
Allophone 
Average gross rent ($)
Average value dwelling ($)
Public transit
Walked
Bicycle
Transportation vehicle driver
Transportation vehicle non-driver
Englis

In [129]:
# Remove white spaces
census_2006.columns = census_2006.columns.str.strip()

In [130]:
# Add year
census_2006 = census_2006.add_suffix('_2006')

# Check
print("Column names:")
for col in census_2006.columns:
    print(col)

Column names:
GeoUID_2006
Area (sq km)_2006
Population_2006
Dwellings_2006
Households_2006
Total male population_2006
Total female population_2006
Never married_2006
Divorced_2006
Widowed_2006
Non-movers_2006
Movers_2006
Non-Canadian citizens_2006
Canadian citizens_2006
Non-migrants_2006
Migrants_2006
Internal migrants_2006
External migrants_2006
Non-immigrants_2006
Immigrants_2006
Visible minority_2006
Non-visible minority_2006
Non-permanent residents_2006
Common-law couples_2006
Married couples_2006
Median household income $_2006
Employed_2006
Unemployed_2006
No certificate or diploma_2006
High school or Secondary degree_2006
College or CEGEP degree_2006
Trades certificate, diploma or apprenticeship_2006
Bachelors degree or higher_2006
Apartment with fewer than five stories_2006
Apartment with five or more storeys_2006
Row house_2006
Single-detached house_2006
Semi-detached house_2006
Movable dwelling_2006
Other single-attached house_2006
Major repairs_2006
Owned housing_2006
Rented 

In [131]:
# Check values
census_2006.iloc[:60, 47:67]

,Average gross rent ($)_2006,Average value dwelling ($)_2006,Public transit_2006,Walked_2006,Bicycle_2006,Transportation vehicle driver_2006,Transportation vehicle non-driver_2006,English and French_2006,English and non-official language(s)_2006,French and non-official language(s)_2006,"English, French, and non-official language(s)_2006",Population 0 to 9_2006,Population 10 to 19_2006,Population 20 to 29_2006,Population 30 to 39_2006,Population 40 to 49_2006,Population 50 to 59_2006,Population 60 plus_2006,Regular maintenance_2006,Transportation other methods_2006
0,599.0,199403.0,360.0,40.0,0.0,750.0,50.0,10.0,0.0,10.0,0.0,205.0,240.0,325.0,345.0,480.0,380.0,520.0,1125.0,10.0
1,592.0,205372.0,505.0,80.0,10.0,710.0,10.0,0.0,0.0,15.0,0.0,250.0,340.0,375.0,395.0,460.0,375.0,620.0,1240.0,0.0
2,591.0,243052.0,905.0,190.0,10.0,1870.0,165.0,30.0,10.0,40.0,10.0,585.0,715.0,860.0,785.0,1090.0,1010.0,1375.0,2665.0,10.0
3,563.0,204537.0,550.0,105.0,40.0,750.0,65.0,10.0,0.0,50.0,10.0,225.0,335.0,430.0,375.0,555.0,510.0,785.0,1455.0,20.0
4,569.0,210709.0,400.0,35.0,35.0,935.0,70.0,20.0,0.0,0.0,10.0,245.0,310.0,400.0,430.0,580.0,515.0,725.0,1455.0,0.0
5,596.0,230752.0,820.0,80.0,30.0,1060.0,80.0,30.0,0.0,40.0,0.0,545.0,580.0,730.0,645.0,785.0,595.0,685.0,1810.0,10.0
6,545.0,197477.0,1090.0,185.0,15.0,1290.0,70.0,25.0,0.0,10.0,0.0,430.0,570.0,760.0,665.0,975.0,905.0,1390.0,2550.0,30.0
7,615.0,204810.0,435.0,60.0,15.0,810.0,70.0,0.0,0.0,25.0,0.0,245.0,345.0,365.0,375.0,580.0,495.0,590.0,1305.0,10.0
8,576.0,211818.0,655.0,90.0,30.0,855.0,45.0,25.0,0.0,25.0,0.0,305.0,405.0,545.0,480.0,630.0,555.0,655.0,1455.0,0.0
9,602.0,181282.0,165.0,35.0,25.0,390.0,25.0,10.0,0.0,15.0,0.0,120.0,105.0,200.0,175.0,230.0,215.0,310.0,585.0,0.0


In [132]:
# Export file to drive
census_2006.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2006_clean.csv', index=False)

# **2011**
- merge age buckets
- rename variables!
- drop "NHS Non Return Rate" column

In [148]:
census_2011 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/2011_data_variables.csv')

# Print all column names
print("Column names:")
for col in census_2011.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
NHS Non Return Rate
v_CA11F_8: 0 to 4 years
v_CA11F_11: 5 to 9 years
v_CA11F_14: 10 to 14 years
v_CA11F_17: 15 to 19 years
v_CA11F_35: 20 to 24 years
v_CA11F_38: 25 to 29 years
v_CA11F_41: 30 to 34 years
v_CA11F_44: 35 to 39 years
v_CA11F_47: 40 to 44 years
v_CA11F_50: 45 to 49 years
v_CA11F_6: Total population by age groups
v_CA11F_7: Total population by age groups
v_CA11F_53: 50 to 54 years
v_CA11F_56: 55 to 59 years
v_CA11F_59: 60 to 64 years
v_CA11F_62: 65 to 69 years
v_CA11F_65: 70 to 74 years
v_CA11F_68: 75 to 79 years
v_CA11F_71: 80 to 84 years
v_CA11F_74: 85 years and over
v_CA11F_98: Single (never legally married)
v_CA11F_104: Divorced
v_CA11F_107: Widowed
v_CA11F_123: Common-law couples
v_CA11F_117: Married couples
v_CA11F_207: Apartment, building that has fewer than five storeys
v_CA11F_201: Apartment, building that has five or more storeys
v_CA11F_205: Row house
v_CA11F_204: Semi-detached 

In [149]:
# Remove white spaces
census_2011.columns = census_2011.columns.str.strip()

In [150]:
# Convert all columns except the identifier to numeric
ctuidcol = 'GeoUID'  # identifier column
cols_to_convert = census_2011.columns.difference([ctuidcol])
census_2011[cols_to_convert] = census_2011[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [151]:
# Merge population age buckets

census_2011['Population 0 to 9'] = census_2011['v_CA11F_8: 0 to 4 years'] + census_2011['v_CA11F_11: 5 to 9 years']
census_2011 = census_2011.drop(columns=['v_CA11F_8: 0 to 4 years', 'v_CA11F_11: 5 to 9 years'])

census_2011['Population 10 to 19'] = census_2011['v_CA11F_14: 10 to 14 years'] + census_2011['v_CA11F_17: 15 to 19 years']
census_2011 = census_2011.drop(columns=['v_CA11F_14: 10 to 14 years', 'v_CA11F_17: 15 to 19 years'])

census_2011['Population 20 to 29'] = census_2011['v_CA11F_35: 20 to 24 years'] + census_2011['v_CA11F_38: 25 to 29 years']
census_2011 = census_2011.drop(columns=['v_CA11F_35: 20 to 24 years', 'v_CA11F_38: 25 to 29 years'])

census_2011['Population 30 to 39'] = census_2011['v_CA11F_41: 30 to 34 years'] + census_2011['v_CA11F_44: 35 to 39 years']
census_2011 = census_2011.drop(columns=['v_CA11F_41: 30 to 34 years', 'v_CA11F_44: 35 to 39 years'])

census_2011['Population 40 to 49'] = census_2011['v_CA11F_47: 40 to 44 years'] + census_2011['v_CA11F_50: 45 to 49 years']
census_2011 = census_2011.drop(columns=['v_CA11F_47: 40 to 44 years', 'v_CA11F_50: 45 to 49 years'])

census_2011['Population 50 to 59'] = census_2011['v_CA11F_53: 50 to 54 years'] + census_2011['v_CA11F_56: 55 to 59 years']
census_2011 = census_2011.drop(columns=['v_CA11F_53: 50 to 54 years', 'v_CA11F_56: 55 to 59 years'])

census_2011['Population 60 plus'] = census_2011['v_CA11F_59: 60 to 64 years'] + census_2011['v_CA11F_62: 65 to 69 years'] + census_2011['v_CA11F_65: 70 to 74 years'] + census_2011['v_CA11F_68: 75 to 79 years'] + census_2011['v_CA11F_71: 80 to 84 years'] + census_2011['v_CA11F_74: 85 years and over']
census_2011 = census_2011.drop(columns=['v_CA11F_59: 60 to 64 years', 'v_CA11F_62: 65 to 69 years', 'v_CA11F_65: 70 to 74 years', 'v_CA11F_68: 75 to 79 years', 'v_CA11F_71: 80 to 84 years', 'v_CA11F_74: 85 years and over'])

In [152]:
 # Calculate pop percentage change and add
c_2006 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2006_clean.csv')
census_2011['Population change (%)'] = ((census_2011['Population'] - c_2006['Population_2006']) / c_2006['Population_2006'] * 100).round(1)

print(census_2011['Population change (%)'])

0      4.4
1      7.8
2     -2.6
3      1.2
4     -0.8
      ... 
916    NaN
917    NaN
918    NaN
919    NaN
920    NaN
Name: Population change (%), Length: 921, dtype: float64


In [153]:
# Drop useless
census_2011 = census_2011.drop(columns=['NHS Non Return Rate', 'Type', 'Region Name'])

# Check
print("Column names:")
for col in census_2011.columns:
    print(col)

Column names:
GeoUID
Area (sq km)
Population
Dwellings
Households
v_CA11F_6: Total population by age groups
v_CA11F_7: Total population by age groups
v_CA11F_98: Single (never legally married)
v_CA11F_104: Divorced
v_CA11F_107: Widowed
v_CA11F_123: Common-law couples
v_CA11F_117: Married couples
v_CA11F_207: Apartment, building that has fewer than five storeys
v_CA11F_201: Apartment, building that has five or more storeys
v_CA11F_205: Row house
v_CA11F_204: Semi-detached house
v_CA11F_202: Movable dwelling
v_CA11F_208: Other single-attached house
v_CA11F_224: English
v_CA11F_227: French
v_CA11F_230: Non-official languages
v_CA11F_200: Single-detached house
v_CA11F_539: English and French
v_CA11F_542: English and non-official language
v_CA11F_545: French and non-official language
v_CA11F_548: English, French and non-official language
v_CA11N_1720: Non-movers
v_CA11N_1723: Movers
v_CA11N_1726: Non-migrants
v_CA11N_1729: Migrants
v_CA11N_1732: Internal migrants
v_CA11N_1741: External migr

In [154]:
# Rename variables!
census_2011 = census_2011.rename(columns=
  {'v_CA11F_6: Total population by age groups': 'Total male population', 'v_CA11F_7: Total population by age groups': 'Total female population',
   'v_CA11N_1720: Non-movers': 'Non-movers', 'v_CA11N_1723: Movers': 'Movers', 'v_CA11N_1726: Non-migrants': 'Non-migrants', 'v_CA11N_1729: Migrants': 'Migrants',
   'v_CA11N_1732: Internal migrants': 'Internal migrants', 'v_CA11N_1741: External migrants': 'External migrants', 'v_CA11N_499: Not a visible minority': 'Non-visible minority',
   'v_CA11N_460: Total visible minority population': 'Visible minority', 'v_CA11N_13: Not Canadian citizens': 'Non-Canadian citizens', 'v_CA11N_4: Canadian citizens': 'Canadian citizens',
   'v_CA11N_19: Non-immigrants': 'Non-immigrants', 'v_CA11N_22: Immigrants': 'Immigrants', 'v_CA11N_46: Non-permanent residents': 'Non-permanent residents',
   'v_CA11F_98: Single (never legally married)': 'Never married', 'v_CA11F_104: Divorced': 'Divorced', 'v_CA11F_107: Widowed': 'Widowed', 'v_CA11F_123: Common-law couples': 'Common-law couples', 'v_CA11F_117: Married couples': 'Married couples',
   'v_CA11N_2562: Median household total income $': 'Median household income ($)', 'v_CA11N_1993: Employed': 'Employed', 'v_CA11N_1996: Unemployed': 'Unemployed',
   'v_CA11N_1804: No certificate, diploma or degree': 'No certificate or diploma', 'v_CA11N_1807: High school diploma or equivalent': 'High school or Secondary degree',
   'v_CA11N_1816: College, CEGEP or other non-university certificate or diploma': 'College or CEGEP degree', 'v_CA11N_1813: Apprenticeship or trades certificate or diploma': 'Trades certificate, diploma or apprenticeship',
   'v_CA11N_1822: University certificate, diploma or degree at bachelor level or above': 'Bachelors degree or higher',
   'v_CA11F_207: Apartment, building that has fewer than five storeys': 'Apartment with fewer than five stories', 'v_CA11F_201: Apartment, building that has five or more storeys': 'Apartment with five or more storeys',
   'v_CA11F_205: Row house': 'Row house', 'v_CA11F_200: Single-detached house': 'Single-detached house', 'v_CA11F_204: Semi-detached house': 'Semi-detached house', 'v_CA11F_202: Movable dwelling': 'Movable dwelling', 'v_CA11F_208: Other single-attached house': 'Other single-attached house',
   'v_CA11N_2292: Average monthly shelter costs for rented dwellings ($)': 'Average gross rent ($)', 'v_CA11N_2287: Average value of dwellings ($)': 'Average value dwelling ($)',
   'v_CA11N_2232: Major repairs needed': 'Major repairs', 'v_CA11N_2231: Only regular maintenance or minor repairs needed': 'Regular maintenance',
   'v_CA11N_2253: Owner': 'Owned housing', 'v_CA11N_2254: Renter': 'Rented housing', 'v_CA11N_2255: Band housing': 'Band housing',

   'v_CA11N_2200: Public transit': 'Public transit', 'v_CA11N_2203: Walked': 'Walked', 'v_CA11N_2206: Bicycle': 'Bicycle',
   'v_CA11N_2209: Other methods': 'Transportation other methods', 'v_CA11N_2194: Car, truck or van - as a driver': 'Transportation vehicle driver', 'v_CA11N_2197: Car, truck or van - as a passenger': 'Transportation vehicle non-driver',
   'v_CA11F_224: English': 'English only', 'v_CA11F_227: French': 'French only', 'v_CA11F_539: English and French': 'English and French',
   'v_CA11F_542: English and non-official language': 'English and non-official language(s)', 'v_CA11F_545: French and non-official language': 'French and non-official language(s)',
   'v_CA11F_548: English, French and non-official language': 'English, French, and non-official language(s)', 'v_CA11F_230: Non-official languages': 'Allophone'
   })

# Check
print("Column names:")
for col in census_2011.columns:
    print(col)

print(len(census_2011.columns))

Column names:
GeoUID
Area (sq km)
Population
Dwellings
Households
Total male population
Total female population
Never married
Divorced
Widowed
Common-law couples
Married couples
Apartment with fewer than five stories
Apartment with five or more storeys
Row house
Semi-detached house
Movable dwelling
Other single-attached house
English only
French only
Allophone
Single-detached house
English and French
English and non-official language(s)
French and non-official language(s)
English, French, and non-official language(s)
Non-movers
Movers
Non-migrants
Migrants
Internal migrants
External migrants
Visible minority
Non-visible minority
Non-Canadian citizens
Canadian citizens
Non-immigrants
Immigrants
Non-permanent residents
Median household income ($)
Employed
Unemployed
No certificate or diploma
High school or Secondary degree
College or CEGEP degree
Trades certificate, diploma or apprenticeship
Bachelors degree or higher
Average gross rent ($)
Average value dwelling ($)
Owned housing
Rented

In [155]:
# Add year
census_2011 = census_2011.add_suffix('_2011')

# Check
print("Column names:")
for col in census_2011.columns:
    print(col)

print(len(census_2011.columns))

Column names:
GeoUID_2011
Area (sq km)_2011
Population_2011
Dwellings_2011
Households_2011
Total male population_2011
Total female population_2011
Never married_2011
Divorced_2011
Widowed_2011
Common-law couples_2011
Married couples_2011
Apartment with fewer than five stories_2011
Apartment with five or more storeys_2011
Row house_2011
Semi-detached house_2011
Movable dwelling_2011
Other single-attached house_2011
English only_2011
French only_2011
Allophone_2011
Single-detached house_2011
English and French_2011
English and non-official language(s)_2011
French and non-official language(s)_2011
English, French, and non-official language(s)_2011
Non-movers_2011
Movers_2011
Non-migrants_2011
Migrants_2011
Internal migrants_2011
External migrants_2011
Visible minority_2011
Non-visible minority_2011
Non-Canadian citizens_2011
Canadian citizens_2011
Non-immigrants_2011
Immigrants_2011
Non-permanent residents_2011
Median household income ($)_2011
Employed_2011
Unemployed_2011
No certificate 

In [159]:
# Check values
census_2011.iloc[:60, 47:67]

,Average gross rent ($)_2011,Average value dwelling ($)_2011,Owned housing_2011,Rented housing_2011,Band housing_2011,Major repairs_2011,Regular maintenance_2011,Walked_2011,Bicycle_2011,Transportation other methods_2011,Public transit_2011,Transportation vehicle driver_2011,Transportation vehicle non-driver_2011,Population 0 to 9_2011,Population 10 to 19_2011,Population 20 to 29_2011,Population 30 to 39_2011,Population 40 to 49_2011,Population 50 to 59_2011,Population 60 plus_2011
0,658.0,285258.0,615.0,700.0,0.0,125.0,1195.0,40.0,25.0,0.0,270.0,940.0,25.0,210.0,240.0,350.0,365.0,380.0,505.0,550.0
1,674.0,403919.0,525.0,1025.0,0.0,145.0,1400.0,65.0,0.0,15.0,470.0,705.0,0.0,285.0,315.0,360.0,420.0,455.0,450.0,760.0
2,673.0,333478.0,1260.0,1615.0,0.0,240.0,2635.0,90.0,50.0,0.0,1075.0,1705.0,135.0,610.0,610.0,785.0,805.0,900.0,1105.0,1430.0
3,610.0,300657.0,615.0,1000.0,0.0,100.0,1515.0,65.0,35.0,0.0,620.0,735.0,35.0,265.0,305.0,420.0,420.0,445.0,620.0,785.0
4,665.0,295716.0,665.0,965.0,0.0,110.0,1520.0,70.0,25.0,0.0,460.0,925.0,85.0,270.0,270.0,395.0,435.0,465.0,555.0,760.0
5,669.0,311808.0,530.0,1485.0,0.0,170.0,1850.0,65.0,0.0,35.0,720.0,1020.0,120.0,505.0,570.0,645.0,625.0,680.0,650.0,795.0
6,676.0,297209.0,1390.0,1430.0,0.0,190.0,2625.0,200.0,50.0,20.0,1110.0,1305.0,105.0,465.0,590.0,720.0,710.0,895.0,1005.0,1325.0
7,671.0,298613.0,675.0,730.0,0.0,135.0,1265.0,35.0,0.0,0.0,565.0,955.0,35.0,295.0,295.0,370.0,395.0,465.0,520.0,600.0
8,692.0,278812.0,700.0,965.0,0.0,165.0,1500.0,145.0,15.0,0.0,550.0,1035.0,40.0,300.0,335.0,550.0,530.0,540.0,570.0,655.0
9,683.0,238311.0,310.0,430.0,0.0,35.0,710.0,15.0,15.0,0.0,195.0,335.0,0.0,135.0,110.0,245.0,200.0,210.0,250.0,295.0


In [157]:
# Export file to drive
census_2011.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2011_clean.csv' , index=False)

# **2016**
- merge age buckets
- rename variables!


In [160]:
census_2016 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/2016_data_variables.csv')

# Print all column names
print("Column names:")
for col in census_2016.columns:
    print(col)


Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA16_7: 0 to 4 years
v_CA16_25: 5 to 9 years
v_CA16_43: 10 to 14 years
v_CA16_2: Age Stats
v_CA16_3: Age Stats
v_CA16_64: 15 to 19 years
v_CA16_82: 20 to 24 years
v_CA16_100: 25 to 29 years
v_CA16_118: 30 to 34 years
v_CA16_136: 35 to 39 years
v_CA16_154: 40 to 44 years
v_CA16_172: 45 to 49 years
v_CA16_190: 50 to 54 years
v_CA16_208: 55 to 59 years
v_CA16_226: 60 to 64 years
v_CA16_247: 65 to 69 years
v_CA16_265: 70 to 74 years
v_CA16_283: 75 to 79 years
v_CA16_301: 80 to 84 years
v_CA16_319: 85 years and over
v_CA16_403: Population percentage change, 2011 to 2016
v_CA16_415: Apartment in a building that has fewer than five storeys
v_CA16_410: Apartment in a building that has five or more storeys
v_CA16_413: Row house
v_CA16_409: Single-detached house
v_CA16_412: Semi-detached house
v_CA16_417: Movable dwelling
v_CA16_416: Other single-attached house
v_CA16_6695: Non-movers
v_CA16_6698: Movers
v_CA

In [161]:
# Remove white spaces
census_2016.columns = census_2016.columns.str.strip()

In [162]:
# Convert all columns except the identifier to numeric
ctuidcol = 'GeoUID'  # identifier column
cols_to_convert = census_2016.columns.difference([ctuidcol])
census_2016[cols_to_convert] = census_2016[cols_to_convert].apply(pd.to_numeric, errors='coerce')


In [163]:
# Merge population age buckets

census_2016['Population 0 to 9'] = census_2016['v_CA16_7: 0 to 4 years'] + census_2016['v_CA16_25: 5 to 9 years']
census_2016 = census_2016.drop(columns=['v_CA16_7: 0 to 4 years', 'v_CA16_25: 5 to 9 years'])

census_2016['Population 10 to 19'] = census_2016['v_CA16_43: 10 to 14 years'] + census_2016['v_CA16_64: 15 to 19 years']
census_2016 = census_2016.drop(columns=['v_CA16_43: 10 to 14 years', 'v_CA16_64: 15 to 19 years'])

census_2016['Population 20 to 29'] = census_2016['v_CA16_82: 20 to 24 years'] + census_2016['v_CA16_100: 25 to 29 years']
census_2016 = census_2016.drop(columns=['v_CA16_82: 20 to 24 years', 'v_CA16_100: 25 to 29 years'])

census_2016['Population 30 to 39'] = census_2016['v_CA16_118: 30 to 34 years'] + census_2016['v_CA16_136: 35 to 39 years']
census_2016 = census_2016.drop(columns=['v_CA16_118: 30 to 34 years', 'v_CA16_136: 35 to 39 years'])

census_2016['Population 40 to 49'] = census_2016['v_CA16_154: 40 to 44 years'] + census_2016['v_CA16_172: 45 to 49 years']
census_2016 = census_2016.drop(columns=['v_CA16_154: 40 to 44 years', 'v_CA16_172: 45 to 49 years'])

census_2016['Population 50 to 59'] = census_2016['v_CA16_190: 50 to 54 years'] + census_2016['v_CA16_208: 55 to 59 years']
census_2016 = census_2016.drop(columns=['v_CA16_190: 50 to 54 years', 'v_CA16_208: 55 to 59 years'])

census_2016['Population 60 plus'] = census_2016['v_CA16_226: 60 to 64 years'] + census_2016['v_CA16_247: 65 to 69 years'] + census_2016['v_CA16_265: 70 to 74 years'] + census_2016['v_CA16_283: 75 to 79 years'] + census_2016['v_CA16_301: 80 to 84 years'] + census_2016['v_CA16_319: 85 years and over']
census_2016 = census_2016.drop(columns=['v_CA16_226: 60 to 64 years', 'v_CA16_247: 65 to 69 years', 'v_CA16_265: 70 to 74 years', 'v_CA16_283: 75 to 79 years', 'v_CA16_301: 80 to 84 years', 'v_CA16_319: 85 years and over'])

In [164]:
# Drop useless
census_2016 = census_2016.drop(columns=['Type','Region Name'])

# Check
print("Column names:")
for col in census_2016.columns:
    print(col)

Column names:
GeoUID
Area (sq km)
Population
Dwellings
Households
v_CA16_2: Age Stats
v_CA16_3: Age Stats
v_CA16_403: Population percentage change, 2011 to 2016
v_CA16_415: Apartment in a building that has fewer than five storeys
v_CA16_410: Apartment in a building that has five or more storeys
v_CA16_413: Row house
v_CA16_409: Single-detached house
v_CA16_412: Semi-detached house
v_CA16_417: Movable dwelling
v_CA16_416: Other single-attached house
v_CA16_6695: Non-movers
v_CA16_6698: Movers
v_CA16_3957: Total visible minority population
v_CA16_3996: Not a visible minority
v_CA16_3402: Not Canadian citizens
v_CA16_3408: Non-immigrants
v_CA16_3411: Immigrants
v_CA16_3435: Non-permanent residents
v_CA16_3393: Canadian citizens
v_CA16_6701: Non-migrants
v_CA16_6704: Migrants
v_CA16_6707: Internal migrants
v_CA16_6716: External migrants
v_CA16_466: Never married
v_CA16_472: Divorced
v_CA16_475: Widowed
v_CA16_487: Common-law couples
v_CA16_486: Married couples
v_CA16_2397: Median total inc

In [165]:
# Rename variables!
census_2016 = census_2016.rename(columns=
  {'v_CA16_2: Age Stats': 'Total male population', 'v_CA16_3: Age Stats': 'Total female population', 'v_CA16_403: Population percentage change, 2011 to 2016': 'Population change (%)',
   'v_CA16_6695: Non-movers': 'Non-movers', 'v_CA16_6698: Movers': 'Movers', 'v_CA16_6701: Non-migrants': 'Non-migrants', 'v_CA16_6704: Migrants': 'Migrants',
   'v_CA16_6707: Internal migrants': 'Internal migrants', 'v_CA16_6716: External migrants': 'External migrants', 'v_CA16_3996: Not a visible minority': 'Non-visible minority',
   'v_CA16_3957: Total visible minority population': 'Visible minority', 'v_CA16_3402: Not Canadian citizens': 'Non-Canadian citizens', 'v_CA16_3393: Canadian citizens': 'Canadian citizens',
   'v_CA16_3408: Non-immigrants': 'Non-immigrants', 'v_CA16_3411: Immigrants': 'Immigrants', 'v_CA16_3435: Non-permanent residents': 'Non-permanent residents',
   'v_CA16_466: Never married': 'Never married', 'v_CA16_472: Divorced': 'Divorced', 'v_CA16_475: Widowed': 'Widowed', 'v_CA16_487: Common-law couples': 'Common-law couples', 'v_CA16_486: Married couples': 'Married couples',
   'v_CA16_2397: Median total income of households in 2015 ($)': 'Median household income ($)', 'v_CA16_5603: Employed': 'Employed', 'v_CA16_5606: Unemployed': 'Unemployed',
   'v_CA16_5099: No certificate, diploma or degree': 'No certificate or diploma', 'v_CA16_5102: Secondary (high) school diploma or equivalency certificate': 'High school or Secondary degree',
   'v_CA16_5117: College, CEGEP or other non-university certificate or diploma': 'College or CEGEP degree', 'v_CA16_5108: Apprenticeship or trades certificate or diploma': 'Trades certificate, diploma or apprenticeship',
   'v_CA16_5123: University certificate, diploma or degree at bachelor level or above': 'Bachelors degree or higher',
   'v_CA16_415: Apartment in a building that has fewer than five storeys': 'Apartment with fewer than five stories', 'v_CA16_410: Apartment in a building that has five or more storeys': 'Apartment with five or more storeys',
   'v_CA16_413: Row house': 'Row house', 'v_CA16_409: Single-detached house': 'Single-detached house', 'v_CA16_412: Semi-detached house': 'Semi-detached house', 'v_CA16_417: Movable dwelling': 'Movable dwelling', 'v_CA16_416: Other single-attached house': 'Other single-attached house',
   'v_CA16_4901: Average monthly shelter costs for rented dwellings ($)': 'Average gross rent ($)', 'v_CA16_4896: Average value of dwellings ($)': 'Average value dwelling ($)',
   'v_CA16_4872: Major repairs needed': 'Major repairs', 'v_CA16_4871: Only regular maintenance or minor repairs needed': 'Regular maintenance',
   'v_CA16_4837: Owner': 'Owned housing', 'v_CA16_4838: Renter': 'Rented housing', 'v_CA16_4839: Band housing': 'Band housing',
   'v_CA16_5801: Public transit': 'Public transit', 'v_CA16_5804: Walked': 'Walked', 'v_CA16_5807: Bicycle': 'Bicycle',
   'v_CA16_5810: Other method': 'Transportation other methods', 'v_CA16_5795: Car, truck, van - as a driver': 'Transportation vehicle driver', 'v_CA16_5798: Car, truck, van - as a passenger': 'Transportation vehicle non-driver',
   'v_CA16_557: English': 'English only', 'v_CA16_560: French': 'French only', 'v_CA16_1343: English and French': 'English and French',
   'v_CA16_1346: English and non-official language': 'English and non-official language(s)', 'v_CA16_1349: French and non-official language': 'French and non-official language(s)',
   'v_CA16_1352: English, French and non-official language': 'English, French, and non-official language(s)', 'v_CA16_563: Non-official languages': 'Allophone'
   })

In [166]:
# Add year
census_2016 = census_2016.add_suffix('_2016')

# Check
print("Column names:")
for col in census_2016.columns:
    print(col)

print(len(census_2016.columns))

Column names:
GeoUID_2016
Area (sq km)_2016
Population_2016
Dwellings_2016
Households_2016
Total male population_2016
Total female population_2016
Population change (%)_2016
Apartment with fewer than five stories_2016
Apartment with five or more storeys_2016
Row house_2016
Single-detached house_2016
Semi-detached house_2016
Movable dwelling_2016
Other single-attached house_2016
Non-movers_2016
Movers_2016
Visible minority_2016
Non-visible minority_2016
Non-Canadian citizens_2016
Non-immigrants_2016
Immigrants_2016
Non-permanent residents_2016
Canadian citizens_2016
Non-migrants_2016
Migrants_2016
Internal migrants_2016
External migrants_2016
Never married_2016
Divorced_2016
Widowed_2016
Common-law couples_2016
Married couples_2016
Median household income ($)_2016
Employed_2016
Unemployed_2016
No certificate or diploma_2016
High school or Secondary degree_2016
College or CEGEP degree_2016
Trades certificate, diploma or apprenticeship_2016
Bachelors degree or higher_2016
Average gross re

In [168]:
# Check values
census_2016.iloc[:60, 47:67]

,Band housing_2016,Public transit_2016,Walked_2016,Bicycle_2016,Transportation other methods_2016,Transportation vehicle driver_2016,Transportation vehicle non-driver_2016,English only_2016,French only_2016,Allophone_2016,English and French_2016,English and non-official language(s)_2016,French and non-official language(s)_2016,"English, French, and non-official language(s)_2016",Population 0 to 9_2016,Population 10 to 19_2016,Population 20 to 29_2016,Population 30 to 39_2016,Population 40 to 49_2016,Population 50 to 59_2016
0,0.0,395.0,30.0,25.0,15.0,865.0,50.0,50.0,2165.0,325.0,20.0,5.0,25.0,5.0,230.0,215.0,365.0,375.0,365.0,485.0
1,0.0,490.0,80.0,20.0,15.0,965.0,85.0,80.0,2880.0,455.0,25.0,5.0,55.0,5.0,410.0,285.0,485.0,530.0,470.0,530.0
2,0.0,885.0,125.0,20.0,15.0,1730.0,35.0,210.0,4540.0,1370.0,55.0,20.0,105.0,15.0,665.0,630.0,780.0,920.0,805.0,1035.0
3,0.0,605.0,60.0,10.0,10.0,835.0,60.0,70.0,2515.0,455.0,20.0,5.0,30.0,5.0,275.0,260.0,430.0,440.0,415.0,545.0
4,0.0,430.0,60.0,20.0,0.0,870.0,45.0,60.0,2620.0,275.0,20.0,5.0,25.0,5.0,300.0,215.0,425.0,425.0,390.0,545.0
5,0.0,745.0,90.0,10.0,10.0,955.0,85.0,100.0,3500.0,730.0,40.0,5.0,60.0,15.0,605.0,465.0,670.0,670.0,570.0,665.0
6,0.0,1115.0,135.0,70.0,0.0,1275.0,70.0,170.0,4390.0,945.0,40.0,5.0,100.0,15.0,545.0,455.0,740.0,795.0,715.0,970.0
7,0.0,530.0,55.0,35.0,20.0,840.0,30.0,85.0,2625.0,260.0,20.0,5.0,15.0,0.0,350.0,240.0,390.0,435.0,395.0,545.0
8,0.0,610.0,115.0,50.0,10.0,905.0,45.0,120.0,2755.0,475.0,30.0,10.0,30.0,10.0,365.0,260.0,595.0,550.0,445.0,595.0
9,0.0,285.0,25.0,35.0,10.0,330.0,15.0,30.0,1165.0,225.0,10.0,5.0,15.0,5.0,170.0,125.0,195.0,245.0,205.0,230.0


In [169]:
# Export file to drive
census_2016.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2016_clean.csv' , index=False)

# **2021**
- merge age buckets
- rename variables!
- merge "Multiple non-official languages" column and "Non-official languages" column

In [186]:
census_2021 = pd.read_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/1_raw_census_data/2021_data_variables.csv')

# Print all column names
print("Column names:")
for col in census_2021.columns:
    print(col)

Column names:
GeoUID
Type
Region Name
Area (sq km)
Population 
Dwellings 
Households 
v_CA21_14: 0 to 4 years
v_CA21_32: 5 to 9 years
v_CA21_50: 10 to 14 years
v_CA21_9: Total - Age
v_CA21_10: Total - Age
v_CA21_3: Population percentage change, 2016 to 2021
v_CA21_71: 15 to 19 years
v_CA21_89: 20 to 24 years
v_CA21_107: 25 to 29 years
v_CA21_125: 30 to 34 years
v_CA21_143: 35 to 39 years
v_CA21_161: 40 to 44 years
v_CA21_179: 45 to 49 years
v_CA21_197: 50 to 54 years
v_CA21_215: 55 to 59 years
v_CA21_233: 60 to 64 years
v_CA21_254: 65 to 69 years
v_CA21_272: 70 to 74 years
v_CA21_290: 75 to 79 years
v_CA21_308: 80 to 84 years
v_CA21_326: 85 years and over
v_CA21_5748: Non-movers
v_CA21_5751: Movers
v_CA21_5754: Non-migrants
v_CA21_5757: Migrants
v_CA21_5760: Internal migrants
v_CA21_5769: External migrants
v_CA21_4875: Total visible minority population
v_CA21_4914: Not a visible minority
v_CA21_4401: Not Canadian citizens
v_CA21_4407: Non-immigrants
v_CA21_4410: Immigrants
v_CA21_4434:

In [187]:
# Remove white spaces
census_2021.columns = census_2021.columns.str.strip()

In [188]:
# Convert all columns except the identifier to numeric
ctuidcol = 'GeoUID'  # identifier column
cols_to_convert = census_2021.columns.difference([ctuidcol])
census_2021[cols_to_convert] = census_2021[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [189]:
# Merge population age buckets

census_2021['Population 0 to 9'] = census_2021['v_CA21_14: 0 to 4 years'] + census_2021['v_CA21_32: 5 to 9 years']
census_2021 = census_2021.drop(columns=['v_CA21_14: 0 to 4 years', 'v_CA21_32: 5 to 9 years'])

census_2021['Population 10 to 19'] = census_2021['v_CA21_50: 10 to 14 years'] + census_2021['v_CA21_71: 15 to 19 years']
census_2021 = census_2021.drop(columns=['v_CA21_50: 10 to 14 years', 'v_CA21_71: 15 to 19 years'])

census_2021['Population 20 to 29'] = census_2021['v_CA21_89: 20 to 24 years'] + census_2021['v_CA21_107: 25 to 29 years']
census_2021 = census_2021.drop(columns=['v_CA21_89: 20 to 24 years', 'v_CA21_107: 25 to 29 years'])

census_2021['Population 30 to 39'] = census_2021['v_CA21_125: 30 to 34 years'] + census_2021['v_CA21_143: 35 to 39 years']
census_2021 = census_2021.drop(columns=['v_CA21_125: 30 to 34 years', 'v_CA21_143: 35 to 39 years'])

census_2021['Population 40 to 49'] = census_2021['v_CA21_161: 40 to 44 years'] + census_2021['v_CA21_179: 45 to 49 years']
census_2021 = census_2021.drop(columns=['v_CA21_161: 40 to 44 years', 'v_CA21_179: 45 to 49 years'])

census_2021['Population 50 to 59'] = census_2021['v_CA21_197: 50 to 54 years'] + census_2021['v_CA21_215: 55 to 59 years']
census_2021 = census_2021.drop(columns=['v_CA21_197: 50 to 54 years', 'v_CA21_215: 55 to 59 years'])

census_2021['Population 60 plus'] = census_2021['v_CA21_233: 60 to 64 years'] + census_2021['v_CA21_254: 65 to 69 years'] + census_2021['v_CA21_272: 70 to 74 years'] + census_2021['v_CA21_290: 75 to 79 years'] + census_2021['v_CA21_308: 80 to 84 years'] + census_2021['v_CA21_326: 85 years and over']
census_2021 = census_2021.drop(columns=['v_CA21_233: 60 to 64 years', 'v_CA21_254: 65 to 69 years', 'v_CA21_272: 70 to 74 years', 'v_CA21_290: 75 to 79 years', 'v_CA21_308: 80 to 84 years', 'v_CA21_326: 85 years and over'])

In [190]:
# Merge non-official languages and multiple non-official languages

census_2021['Allophone'] = census_2021['v_CA21_2215: Non-official languages'] + census_2021['v_CA21_3190: Multiple non-official languages']
census_2021 = census_2021.drop(columns=['v_CA21_2215: Non-official languages', 'v_CA21_3190: Multiple non-official languages'])

In [191]:
# Drop useless
census_2021 = census_2021.drop(columns=['Type','Region Name'])

# Check
print("Column names:")
for col in census_2021.columns:
    print(col)

Column names:
GeoUID
Area (sq km)
Population
Dwellings
Households
v_CA21_9: Total - Age
v_CA21_10: Total - Age
v_CA21_3: Population percentage change, 2016 to 2021
v_CA21_5748: Non-movers
v_CA21_5751: Movers
v_CA21_5754: Non-migrants
v_CA21_5757: Migrants
v_CA21_5760: Internal migrants
v_CA21_5769: External migrants
v_CA21_4875: Total visible minority population
v_CA21_4914: Not a visible minority
v_CA21_4401: Not Canadian citizens
v_CA21_4407: Non-immigrants
v_CA21_4410: Immigrants
v_CA21_4434: Non-permanent residents
v_CA21_4392: Canadian citizens
v_CA21_480: Never married
v_CA21_486: Divorced
v_CA21_489: Widowed
v_CA21_504: Common-law couples
v_CA21_501: Married couples
v_CA21_906: Median total income of household in 2020 ($)
v_CA21_6498: Employed
v_CA21_6501: Unemployed
v_CA21_5868: No certificate, diploma or degree
v_CA21_5889: College, CEGEP or other non-university certificate or diploma
v_CA21_5880: Apprenticeship or trades certificate or diploma
v_CA21_5895: Bachelor's degree o

In [192]:
census_2021 = census_2021.rename(columns=
  {'v_CA21_9: Total - Age': 'Total male population', 'v_CA21_10: Total - Age': 'Total female population', 'v_CA21_3: Population percentage change, 2016 to 2021': 'Population change (%)',
   'v_CA21_5748: Non-movers': 'Non-movers', 'v_CA21_5751: Movers': 'Movers', 'v_CA21_5754: Non-migrants': 'Non-migrants', 'v_CA21_5757: Migrants': 'Migrants',
   'v_CA21_5760: Internal migrants': 'Internal migrants', 'v_CA21_5769: External migrants': 'External migrants',
   'v_CA21_4914: Not a visible minority': 'Non-visible minority', 'v_CA21_4875: Total visible minority population': 'Visible minority',
   'v_CA21_4401: Not Canadian citizens': 'Non-Canadian citizens', 'v_CA21_4392: Canadian citizens': 'Canadian citizens',
   'v_CA21_4407: Non-immigrants': 'Non-immigrants', 'v_CA21_4410: Immigrants': 'Immigrants', 'v_CA21_4434: Non-permanent residents': 'Non-permanent residents',
   'v_CA21_480: Never married': 'Never married', 'v_CA21_486: Divorced': 'Divorced', 'v_CA21_489: Widowed': 'Widowed', 'v_CA21_504: Common-law couples': 'Common-law couples', 'v_CA21_501: Married couples': 'Married couples',
   'v_CA21_906: Median total income of household in 2020 ($)': 'Median household income ($)', 'v_CA21_6498: Employed': 'Employed', 'v_CA21_6501: Unemployed': 'Unemployed',
   'v_CA21_5868: No certificate, diploma or degree': 'No certificate or diploma', 'v_CA21_5814: With high school diploma or equivalency certificate': 'High school or Secondary degree',
   'v_CA21_5889: College, CEGEP or other non-university certificate or diploma': 'College or CEGEP degree', 'v_CA21_5880: Apprenticeship or trades certificate or diploma': 'Trades certificate, diploma or apprenticeship',
   "v_CA21_5895: Bachelor's degree or higher": "Bachelors degree or higher",
   'v_CA21_439: Apartment in a building that has fewer than five storeys': 'Apartment with fewer than five stories', 'v_CA21_440: Apartment in a building that has five or more storeys': 'Apartment with five or more storeys',
   'v_CA21_437: Row house': 'Row house', 'v_CA21_435: Single-detached house': 'Single-detached house', 'v_CA21_436: Semi-detached house': 'Semi-detached house', 'v_CA21_442: Movable dwelling': 'Movable dwelling', 'v_CA21_441: Other single-attached house': 'Other single-attached house',
   'v_CA21_4318: Average monthly shelter costs for rented dwellings ($) (59)': 'Average gross rent ($)', 'v_CA21_4312: Average value of dwellings ($) (60)': 'Average value dwelling ($)',
   'v_CA21_4274: Major repairs needed': 'Major repairs', 'v_CA21_4273: Only regular maintenance and minor repairs needed': 'Regular maintenance',
   'v_CA21_4238: Owner': 'Owned housing', 'v_CA21_4239: Renter': 'Rented housing', 'v_CA21_4240: Dwelling provided by the local government, First Nation or Indian band': 'Band housing',
   'v_CA21_7644: Public transit': 'Public transit', 'v_CA21_7647: Walked': 'Walked', 'v_CA21_7650: Bicycle': 'Bicycle',
   'v_CA21_7653: Other method': 'Transportation other methods', 'v_CA21_7638: Car, truck or van - as a driver': 'Transportation vehicle driver', 'v_CA21_7641: Car, truck or van - as a passenger': 'Transportation vehicle non-driver',
   'v_CA21_2209: English': 'English only', 'v_CA21_2212: French': 'French only', 'v_CA21_3178: English and French': 'English and French',
   'v_CA21_3181: English and non-official language(s)': 'English and non-official language(s)', 'v_CA21_3184: French and non-official language(s)': 'French and non-official language(s)',
   'v_CA21_3187: English, French and non-official language(s)': 'English, French, and non-official language(s)'
   })

In [193]:
# Add year
census_2021 = census_2021.add_suffix('_2021')

# Check
print("Column names:")
for col in census_2021.columns:
    print(col)

print(len(census_2021.columns))

Column names:
GeoUID_2021
Area (sq km)_2021
Population_2021
Dwellings_2021
Households_2021
Total male population_2021
Total female population_2021
Population change (%)_2021
Non-movers_2021
Movers_2021
Non-migrants_2021
Migrants_2021
Internal migrants_2021
External migrants_2021
Visible minority_2021
Non-visible minority_2021
Non-Canadian citizens_2021
Non-immigrants_2021
Immigrants_2021
Non-permanent residents_2021
Canadian citizens_2021
Never married_2021
Divorced_2021
Widowed_2021
Common-law couples_2021
Married couples_2021
Median household income ($)_2021
Employed_2021
Unemployed_2021
No certificate or diploma_2021
College or CEGEP degree_2021
Trades certificate, diploma or apprenticeship_2021
Bachelors degree or higher_2021
High school or Secondary degree_2021
Apartment with fewer than five stories_2021
Apartment with five or more storeys_2021
Row house_2021
Single-detached house_2021
Semi-detached house_2021
Movable dwelling_2021
Other single-attached house_2021
Average gross re

In [194]:
# Check values
census_2021.iloc[:60, 47:67]

,Band housing_2021,Public transit_2021,Walked_2021,Bicycle_2021,Transportation vehicle driver_2021,Transportation vehicle non-driver_2021,Transportation other methods_2021,English only_2021,French only_2021,English and French_2021,English and non-official language(s)_2021,French and non-official language(s)_2021,"English, French, and non-official language(s)_2021",Population 0 to 9_2021,Population 10 to 19_2021,Population 20 to 29_2021,Population 30 to 39_2021,Population 40 to 49_2021,Population 50 to 59_2021,Population 60 plus_2021
0,0.0,170.0,15.0,20.0,765.0,55.0,10.0,115.0,2165.0,60.0,10.0,85.0,10.0,250.0,200.0,375.0,440.0,335.0,400.0,675.0
1,0.0,365.0,30.0,10.0,935.0,45.0,0.0,120.0,3190.0,60.0,5.0,95.0,10.0,440.0,345.0,470.0,605.0,510.0,525.0,890.0
2,0.0,725.0,135.0,50.0,1460.0,120.0,35.0,355.0,5045.0,115.0,40.0,230.0,40.0,775.0,630.0,815.0,985.0,885.0,875.0,1640.0
3,0.0,400.0,65.0,15.0,725.0,30.0,10.0,130.0,2760.0,55.0,10.0,75.0,10.0,330.0,280.0,435.0,510.0,445.0,470.0,885.0
4,0.0,365.0,80.0,15.0,780.0,40.0,45.0,80.0,2790.0,45.0,0.0,65.0,10.0,340.0,270.0,370.0,515.0,435.0,425.0,815.0
5,0.0,620.0,120.0,10.0,940.0,70.0,25.0,145.0,3670.0,95.0,15.0,160.0,25.0,635.0,495.0,700.0,690.0,620.0,550.0,920.0
6,0.0,695.0,120.0,25.0,1100.0,75.0,10.0,250.0,4520.0,105.0,25.0,150.0,55.0,530.0,450.0,700.0,805.0,780.0,865.0,1535.0
7,0.0,305.0,55.0,55.0,745.0,35.0,30.0,135.0,2630.0,60.0,15.0,65.0,5.0,305.0,305.0,395.0,450.0,485.0,455.0,710.0
8,0.0,485.0,75.0,25.0,800.0,65.0,25.0,170.0,2800.0,70.0,15.0,95.0,20.0,380.0,275.0,600.0,615.0,490.0,470.0,740.0
9,0.0,165.0,60.0,15.0,325.0,25.0,10.0,80.0,1180.0,25.0,5.0,25.0,5.0,160.0,120.0,220.0,295.0,195.0,200.0,310.0


In [196]:
# Export file to drive
census_2021.to_csv('/content/drive/MyDrive/GentrificAItion/montreal_data_processing/data_cleaning/2_clean_data/2021_clean.csv' , index=False)